In [1]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import glob
import time
import logging
from rasterstats import zonal_stats
import re

import itertools
from collections import Counter
import gc
from numba import njit

# File System Functions

In [3]:
import os
import glob

# Function to find the tif files in a given folder¶
def find_files_in_folder(folder_path, extension=None, recursive=False):
    matched_files = []
    # Determine the search pattern based on whether an extension is provided and recursion is enabled
    if extension:
        if recursive:
            search_pattern = os.path.join(folder_path, f"**/*.{extension}")
        else:
            search_pattern = os.path.join(folder_path, f"*.{extension}")
    else:
        # No extension specified, handle both recursive and non-recursive cases
        if recursive:
            search_pattern = os.path.join(folder_path, "**/*")
        else:
            search_pattern = os.path.join(folder_path, "*")
    # Use glob to find matching files in the specified directory and subdirectories if recursive
    matched_files.extend(glob.glob(search_pattern, recursive=recursive))
    # If no files are found, return a list with an empty string
    if not matched_files:
        matched_files = [""]
    return matched_files
    

In [4]:
import os

def add_suffix_to_file_name(file_path, suffix):
    """
    Adds a suffix to the file name in the given file path.
    
    Parameters:
    - file_path (str): The original file path.
    - suffix (str): The suffix to add to the file name (before the file extension).
    
    Returns:
    - str: The updated file path with the suffix added to the file name.
    """
    # Separate the directory, file name, and extension
    directory, file_name = os.path.split(file_path)
    base_name, ext = os.path.splitext(file_name)
    
    # Add the suffix to the file name
    new_file_name = f"{base_name}{suffix}{ext}"
    
    # Reconstruct the full file path
    new_file_path = os.path.join(directory, new_file_name)
    return new_file_path

In [6]:
import time

def precise_timing_decorator(func):
    """Decorator for high-precision execution timing."""
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        print(f"⏳ Execution time of {func.__name__}: {end_time - start_time:.6f} seconds")
        return result
    return wrapper


In [12]:
FONT_STYLES = {
    "default": {
        "id_fontsize": 14,
        "id_fontweight": "normal",
        "id_fontcolor": "black",
        "id_fontfamily": "sans-serif",
        "id_bbox": False  # No bounding box
    },
    "bold_high_contrast": {
        "id_fontsize": 16,
        "id_fontweight": "bold",
        "id_fontcolor": "white",
        "id_fontfamily": "Arial",
        "id_bbox": {"facecolor": "black", "alpha": 0.7}  # Dark background for visibility
    },
    "large_minimal": {
        "id_fontsize": 18,
        "id_fontweight": "light",
        "id_fontcolor": "yellow",
        "id_fontfamily": "monospace",
        "id_bbox": None  # No bounding box
    },
    "compact_subtle": {
        "id_fontsize": 12,
        "id_fontweight": "medium",
        "id_fontcolor": "darkred",
        "id_fontfamily": "serif",
        "id_bbox": {"facecolor": "white", "alpha": 0.5}  # Light background for soft contrast
    }
}

In [14]:
@precise_timing_decorator
def plot_with_highlighted_ortho_geometry(
    shapefile, highlight_id, status, processed_ids, 
    ortho_path=None, highlight_color='khaki', edgewidth=0.25, 
    default_color='lightslategrey', opacity=0.4, 
    plot_size=(5, 5), show_ids=True, 
    zoom=True, zoom_out_factor=6, 
    font_style="bold_high_contrast",
    id_bbox = False, project_name = "None", flight = "None"
):

    """
    Plots a GeoDataFrame with different colors for various stages of processing.
    Optionally overlays the plot on an orthomosaic.
    
    Parameters:
    - shapefile (GeoDataFrame): The input GeoDataFrame containing geometries.
    - highlight_id (int or str): The ID of the geometry to highlight.
    - status (str): The processing status: 'processing', 'processed', or 'unprocessed'.
    - processed_ids (list): List of IDs of already processed polygons.
    - ortho_path (str, optional): Path to the orthomosaic GeoTIFF file.
    - highlight_color (str): Color for the highlighted geometry's boundary (default: 'red').
    - edgewidth (float): Width of the boundary line for the highlighted geometry (default: 3).
    - default_color (str): Default color for other geometries (default: 'lightblue').
    - opacity (float): Opacity level of the polygons (default: 0.5).
    - plot_size (tuple): Size of the plot in inches (default: (10, 10)).
    - show_ids (bool): Whether to display the polygon IDs inside the plot (default: True).
    - zoom (bool): Whether to zoom in on the highlighted geometry (default: True).
    - zoom_out_factor (float): Controls zoom-out level (default: 1.5, where 1 = exact bounds).
    - font_style (str): Preset style for ID text. Options:
        - "default": Standard black text with no background.
        - "bold_high_contrast": Large, bold white text with a black background.
        - "large_minimal": Large yellow text with no background.
        - "compact_subtle": Small, dark red text with a light background.
      (default: "default")
    - id_bbox (bool): Whether to show a white bounding box around ID text (default: False).

    """
    # Clear the current figure
    plt.clf()

    # Create a new figure with user-defined size
    fig, ax = plt.subplots(figsize=plot_size)

    # Plot the orthomosaic if provided
    if ortho_path:
        with rasterio.open(ortho_path) as src:
            show(src, ax=ax, title= f"Raster processing status for {project_name}", adjust='box')

            # Reproject the shapefile to match the orthomosaic CRS if necessary
            if shapefile.crs != src.crs:
                print(f"Reprojecting shapefile to match orthomosaic CRS: {src.crs}")
                shapefile = shapefile.to_crs(src.crs)

    # Set default zoom boundaries (entire dataset)
    minx, miny, maxx, maxy = shapefile.total_bounds  
    
    # If zoom is enabled, adjust bounds to zoom in on the highlighted geometry
    if zoom and highlight_id in shapefile['id'].values:
        highlight_geom = shapefile[shapefile['id'] == highlight_id].geometry.iloc[0]
        minx, miny, maxx, maxy = highlight_geom.bounds

        # Apply zoom out factor (expand bounds)
        width = maxx - minx
        height = maxy - miny
        minx -= width * (zoom_out_factor - 1) / 2
        maxx += width * (zoom_out_factor - 1) / 2
        miny -= height * (zoom_out_factor - 1) / 2
        maxy += height * (zoom_out_factor - 1) / 2

        ax.set_xlim(minx, maxx)
        ax.set_ylim(miny, maxy)

    # Plot each polygon with different colors based on processing status
    for idx, geom in shapefile.iterrows():
        if geom['id'] in processed_ids:
            color = 'green'  # Processed
        elif geom['id'] == highlight_id:
            color = highlight_color  # Currently processing
        else:
            color = default_color  # Unprocessed

        # Plot the individual geometry with opacity control
        shapefile.loc[[idx]].plot(ax=ax, edgecolor='black', facecolor=color, linewidth=edgewidth, alpha=opacity)

        # Get font settings from dictionary
        font_settings = FONT_STYLES.get(font_style, FONT_STYLES["default"])  # Default if not found
    
        # Extract font properties
        id_fontweight = font_settings["id_fontweight"]
        id_fontcolor = font_settings["id_fontcolor"]
        id_fontfamily = font_settings["id_fontfamily"]
        if id_bbox:
            id_bbox = font_settings["id_bbox"]

        # Adjust font color based on processing status
        if geom['id'] in processed_ids:
            id_fontcolor = 'lime'  # Processed
        elif geom['id'] == highlight_id:
            id_fontcolor = 'yellow'  # Being processed

        # Adjust font size based on the zoom level or polygon size
        if zoom:
            # Calculate font size based on the zoomed area
            area = (maxx - minx) * (maxy - miny)  # Area of the zoomed-in region
            id_fontsize = area ** 0.5 / 10  # Use square root of area for better scaling
            id_fontsize = max(15, min(20, id_fontsize))  # Limit font size to a reasonable range
        else:
            # Calculate font size based on the polygon size (width or height)
            width = maxx - minx
            height = maxy - miny
            area = width * height
            id_fontsize = area ** 0.5 / 10  # Square root of the area for proportionate size
            id_fontsize = max(10, min(20, id_fontsize))  # Limit font size to a reasonable range

        if show_ids:
            centroid = geom.geometry.centroid
            # Only print IDs that are within the zoomed/cropped area
            if not zoom or (minx <= centroid.x <= maxx and miny <= centroid.y <= maxy):
                ax.text(
                    centroid.x, centroid.y, str(geom['id']), 
                    fontsize=id_fontsize, fontweight=id_fontweight, 
                    color=id_fontcolor, family=id_fontfamily, 
                    ha='center', va='center',
                    bbox=id_bbox if id_bbox else None
                    )

    # Set the title
    ax.set_title(f"Processing Plot #: {highlight_id}" + "\n" +  f"for field {flight}", fontsize=14)

    # Display the plot
    plt.xlabel("Longitude", fontsize=12)
    plt.ylabel("Latitude", fontsize=12)
    plt.show()

In [16]:
## Reduce RAM Usage

In [18]:
import gc

def clean_memory():
    """Clears memory manually."""
    gc.collect()
    print("✅ Cleared unused memory.")


# Getting project files dict

In [89]:
def get_project_files(src_folder, project_name, flight_type='MS'):
    """
    Retrieves orthomosaic and DSM/DTM file paths, organized by flight folder.

    Parameters:
    - src_folder (str): Root directory where projects are stored.
    - project_name (str): Name of the project folder.
    - flight_type (str): Flight type, either 'MS' or '3D'.

    Returns:
    - ortho_dict (dict): { flight_folder_name: [list of orthomosaic files] }
    - dsm_dtm_dict (dict): { flight_folder_name: [list of DSM/DTM files] }
    """
    
    print(f"Fetching file lists for project: {project_name}, flight type: {flight_type}")

    # Define base path for the flight type
    flight_type_folder = os.path.join(src_folder, project_name, flight_type)

    # Find all subfolders within the flight type folder (each representing a flight)
    flight_folders = [
        f for f in os.listdir(flight_type_folder)
        if os.path.isdir(os.path.join(flight_type_folder, f))
    ]

    # Initialize dictionaries
    ortho_dict = {}
    dsm_dtm_dict = {}

    # Loop through each flight folder
    for flight_folder in flight_folders:
        flight_path = os.path.join(flight_type_folder, flight_folder)
        ortho_folder = os.path.join(flight_path, "2_Orthomosaics")
        dsm_dtm_folder = os.path.join(flight_path, "3_DSM_DTM_Elevation_Models")

        # Collect orthomosaic files
        if os.path.exists(ortho_folder):
            ortho_files = [os.path.join(ortho_folder, f) for f in os.listdir(ortho_folder) if f.endswith(".tif")]
            if ortho_files:
                ortho_dict[flight_folder] = ortho_files
        elif 'MS' in flight_folder:
            print(f"Warning: Orthomosaics folder not found in {flight_folder}")

        # Collect DSM/DTM files
        if os.path.exists(dsm_dtm_folder):
            dsm_dtm_files = [os.path.join(dsm_dtm_folder, f) for f in os.listdir(dsm_dtm_folder) if f.endswith(".tif")]
            if dsm_dtm_files:
                dsm_dtm_dict[flight_folder] = dsm_dtm_files
        elif '3D' in flight_folder:
            print(f"Warning: DSM/DTM folder not found in {flight_folder}")

    return ortho_dict, dsm_dtm_dict


# Ensure data is complete

In [133]:
from collections import Counter

@precise_timing_decorator
def check_data_completeness(ortho_dict, dsm_dtm_dict):
    """
    Checks if all required orthomosaic and DSM/DTM files are present for each flight folder,
    ensuring consistency in flight types (either "MS" or "3D").

    Parameters:
    - ortho_dict (dict): Dictionary {flight_folder: list_of_orthomosaics}.
    - dsm_dtm_dict (dict): Dictionary {flight_folder: list_of_dsm_dtm_files}.

    Returns:
    - completeness_status (dict): Summary of completeness for each flight.
    - missing_files_report (dict): Flights with missing data and missing filenames.
    """

    completeness_status = {}
    missing_files_report = {}

    # Define required file suffixes for MS and 3D projects
    required_files_ms = {
        "_index_green_green.tif",
        "_index_ndvi.tif",
        "_index_nir_nir.tif",
        "_index_red_edge_red_edge.tif",
        "_index_red_red.tif",
        "_transparent_mosaic_group1.tif"
    }
    optional_blue_band = "_index_blue_blue.tif"

    # # For older datasets. Remove Later
    # # Define required file suffixes for MS and 3D projects
    # required_files_ms = {
    #     "_transparent_reflectance_green.tif",
    #     "_index_ndvi.tif",
    #     "_transparent_reflectance_nir.tif",
    #     "_transparent_reflectance_red_edge.tif",
    #     "_transparent_reflectance_red.tif",
    # }
    # optional_blue_band = "_transparent_reflectance_blue.tif"
    

    required_files_3d = {
        "_dtm.tif",
        "_dsm.tif"
    }

    # Identify flight types from folder names
    flight_types = {}
    for flight in list(ortho_dict.keys()) + list(dsm_dtm_dict.keys()):
        if "MS" in flight.upper():
            flight_types[flight] = "MS"
        elif "3D" in flight.upper():
            flight_types[flight] = "3D"
        else:
            flight_types[flight] = "UNKNOWN"

    # Count occurrences of each flight type
    type_counts = Counter(flight_types.values())

    # If there are mixed flight types, identify the minority group and warn the user
    if len(type_counts) > 1:
        print("\n⚠️ WARNING: Inconsistent flight types detected!")
        print("Detected flight types:", type_counts)

        # Find the minority group (least frequent flight type)
        minority_type = min(type_counts, key=type_counts.get)
        minority_flights = [flight for flight, f_type in flight_types.items() if f_type == minority_type]

        print(f"🚨 The following flights are of type '{minority_type}', which may be misplaced:")
        print("\n".join(minority_flights))
        print("Please verify if these are in the correct location.\n")

    # Perform completeness checks
    for flight, f_type in flight_types.items():
        if f_type == "MS":
            # Check for missing MS files
            files = ortho_dict.get(flight, [])
            missing_files = [f for f in required_files_ms if not any(f in file for file in files)]

            # Check for blue band consistency
            blue_band_found = any(optional_blue_band in file for file in files)

            if not missing_files:
                completeness_status[flight] = "✅ Complete"
            else:
                completeness_status[flight] = "❌ Incomplete"
                missing_files_report[flight] = missing_files

        elif f_type == "3D":
            # Check for missing 3D files
            files = dsm_dtm_dict.get(flight, [])
            missing_files = [f for f in required_files_3d if not any(f in file for file in files)]

            if not missing_files:
                completeness_status[flight] = "✅ Complete"
            else:
                completeness_status[flight] = "❌ Incomplete"
                missing_files_report[flight] = missing_files

        else:
            print(f"⚠️ Unknown flight type for {flight}. Skipping data check.")

    return completeness_status, missing_files_report


## Drop incomplete projects

In [27]:
import csv
import os

def drop_incomplete_flights(ortho_dict, dsm_dtm_dict, missing, output_csv_folder):
    """
    Drops flights with missing files from the ortho_dict and dsm_dtm_dict.
    
    Args:
        ortho_dict (dict): Dictionary mapping flight names to orthomosaic file paths.
        dsm_dtm_dict (dict): Dictionary mapping flight names to DSM and DTM file paths.
        missing (dict): Dictionary listing missing files for each flight.
    
    Returns:
        ortho_dict (dict): Updated ortho_dict with incomplete flights removed.
        dsm_dtm_dict (dict): Updated dsm_dtm_dict with incomplete flights removed.
    """
    for flight in list(missing.keys()):  # Use list() to avoid modifying dict during iteration
        if flight in ortho_dict:
            del ortho_dict[flight]
        if flight in dsm_dtm_dict:
            del dsm_dtm_dict[flight]

    # write_missing_projects to csv
    csv_log = os.path.join(output_csv_folder, "Incomplete Flights Log.csv")
    write_missing_projects_to_csv(missing, csv_log)

    return ortho_dict, dsm_dtm_dict

def write_missing_projects_to_csv(missing_projects, csv_file):
    """
    Write the missing projects (with missing files) to a CSV file.
    Each missing file will be logged as a separate row along with the project ID and a note.
    """
    # Ensure the directory exists
    output_dir = os.path.dirname(csv_file)
    os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist
    
    # Check if the CSV file already exists
    file_exists = os.path.isfile(csv_file)
    
    # Open the CSV file in append mode
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['id', 'missing_file', 'status'])
        
        # Write the header if the file does not exist
        if not file_exists:
            writer.writeheader()

        # Write each missing project and its corresponding missing files to the CSV
        for project_id, missing_files in missing_projects.items():
            for missing_file in missing_files:
                writer.writerow({
                    'id': project_id,
                    'missing_file': missing_file,
                    'status': 'Incomplete'  # You can change this if you want a different status
                })

In [30]:
def extract_common_prefix(file_paths):
    """Finds the common prefix among a list of file names."""
    common_prefix = os.path.commonprefix([os.path.basename(p) for p in file_paths])
    return re.sub(r'[^a-zA-Z0-9_-]', '', common_prefix.replace(" ", "_"))  # Clean any special characters

# Downscale tiff

In [35]:
from rasterio.enums import Resampling
def downscale_image(file_path, output_folder, scale_factor):
    """
    Downscale a TIFF/JPG file and save it in the structured output directory.

    Parameters:
        file_path (str): The path to the original file.
        output_folder (str): The base output folder.
        scale_factor (float): Scaling factor (e.g., 0.25 for 25%).

    Returns:
        str: Path to the saved downscaled file.
    """
    print(f"\n⚙️ Processing: {os.path.basename(file_path)} at {int(scale_factor * 100)}% resolution")

    try:
        with rasterio.open(file_path) as src:
            # Compute new dimensions
            new_width = int(src.width * scale_factor)
            new_height = int(src.height * scale_factor)
            print(f"   - Original size: {src.width} x {src.height}")
            print(f"   - New size: {new_width} x {new_height}")

            # Read and resample image
            resampled_array = src.read(
                out_shape=(src.count, new_height, new_width),
                resampling=Resampling.bilinear
            )

            # Update metadata
            profile = src.profile
            profile.update(
                width=new_width,
                height=new_height,
                transform=src.transform * src.transform.scale(
                    (src.width / new_width), (src.height / new_height)
                )
            )


            # Save downscaled image
            print(f"   - Saving downscaled file: {output_folder}")
            with rasterio.open(output_folder, "w", **profile) as dst:
                dst.write(resampled_array)

            print(f"✅ Successfully downscaled: {os.path.basename(file_path)}")
            return output_folder

    except Exception as e:
        print(f"❌ ERROR processing {file_path}: {str(e)}")
        return None

# Data Extraction function

In [125]:
# CHECK WHAT DOES IT DO WITH NDVI TIF if given in input rasters

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Column order as specified
STAT_ORDER = [
    "count", "sum", "mean", "median", "std", "min", "max", "range", "minority", "majority", "variety", 
    "variance", "cv", "skewness", "kurtosis", "top_10", "top_15", "top_20", "top_25", "top_35", "top_50", 
    "q25", "q75", "iqr"
]
MORE_STAT_PERCENT = [5, 10, 15, 25, 35, 50]

# Raster sorting order
RASTER_ORDER_ = ["blue", "green", "ndvi", "nir", "red", "rededge"]
RASTER_NAME_KEYS_ = ["blue_blue", "green_green", "nir_nir", "red_red", "red_edge_red_edge", "ndvi_ndvi"]
# RASTER_NAME_KEYS_ = ["reflectance_blue", "reflectance_green", "reflectance_nir", "reflectance_red", "reflectance_red_edge", "reflectance_ndvi"]

def match_raster_keys(order_list, name_keys):
    """
    Maps raster order names to their corresponding file name keys by sorting the name_keys and normalizing both lists.
    Parameters:
        order_list (list): List of raster order names.
        name_keys (list): List of raster file name keys.
    Returns:
        dict: Dictionary mapping order names to matching file name keys.
    """
    sorted_keys = sorted(name_keys, key=len, reverse=False)  # Sort longest names first

    def normalize(name):
        return re.sub(r'[^a-zA-Z0-9]', '', name.lower())  # Remove special characters

    normalized_keys = {normalize(key): key for key in sorted_keys}  # Map normalized names to original names
    mapping = {}

    for order in order_list:
        norm_order = normalize(order)
        for norm_key, original_key in normalized_keys.items():
            if norm_order in norm_key:  # Exact match after stripping characters
                mapping[order] = original_key
                break
    return mapping

RASTER_ORDER = match_raster_keys(RASTER_ORDER_, RASTER_NAME_KEYS_)

def ensure_crs_alignment(gdf, raster_crs):
    """Ensures the GeoDataFrame has the same CRS as the raster."""
    try:
        if gdf.crs != raster_crs:
            logging.warning("📌 CRS mismatch detected. Reprojecting shapefile to match raster CRS.")
            print("📌 CRS mismatch detected. Reprojecting shapefile to match raster CRS.")
            print(f"📌 GEOJSON CRS {gdf.crs} to Raster CRS {raster_crs}.")

            gdf = gdf.to_crs(raster_crs)
    except Exception as e:
        logging.error(f"Error while aligning CRS: {e}")
        raise
    return gdf

def find_raster_files(paths, RASTER_ORDER):
    """Identifies raster files based on naming conventions."""
    rasters = {name: next((tif for tif in paths if name_pattern in tif), None) for name, name_pattern in RASTER_ORDER.items()}
    return {k: v for k, v in rasters.items() if v is not None}

# Ensure this function remains unchanged
# Optimized NDVI Computation Using Numba
@njit
def compute_ndvi_numba(nir, red):
    """Fast NDVI computation using Numba."""
    return np.where((nir + red) == 0, np.nan, (nir - red) / (nir + red))

def generate_ndvi_GeoTIFF_and_values(nir_tif, red_tif, output_tif):
    """Generates an NDVI orthomosaic and calculates values."""
    with rasterio.open(nir_tif) as nir_src, rasterio.open(red_tif) as red_src:
        # Read the data
        nir = nir_src.read(1).astype(np.float32)
        red = red_src.read(1).astype(np.float32)
        transform = nir_src.transform
        crs = nir_src.crs

        # Compute NDVI
        ndvi = compute_ndvi_numba(nir, red)

        # Save NDVI to a GeoTIFF
        profile = nir_src.profile
        profile.update(dtype=rasterio.float32, count=1, compress="lzw")

        with rasterio.open(output_tif, "w", **profile) as dst:
            dst.write(ndvi, 1)

    # Compute NDVI statistics (flatten non-NaN values)
    ndvi_values = ndvi[~np.isnan(ndvi)].flatten()

    return output_tif, ndvi

def compute_additional_statistics(values, MORE_STAT_PERCENT):
    """Computes variance, CV, skewness, kurtosis, variety, and percentile-based stats.
        Parameters:
        values (array-like): List or array of raster values.
        MORE_STAT_PERCENT (list) : List of additional stats percentages
    Returns:
        dict: Dictionary of computed statistics.
    """
    percent_stats = list(itertools.chain.from_iterable([[f"top_{percent}_mean", f"top_{percent}_median", f"top_{percent}_std"] for percent in MORE_STAT_PERCENT]))

    try:
        if len(values) == 0:
            return {stat: np.nan for stat in STAT_ORDER + percent_stats if stat not in STAT_ORDER[:10]}

        values = np.asarray(values).flatten()  # Ensure it's a NumPy array

        stats = {
            "variety": len(np.unique(values)),
            "variance": np.nanvar(values),
            "cv": np.nanstd(values) / np.nanmean(values) if np.nanmean(values) != 0 else np.nan,
            "skewness": pd.Series(values).skew(),
            "kurtosis": pd.Series(values).kurtosis(),
            "top_10": np.nanpercentile(values, 90),
            "top_15": np.nanpercentile(values, 85),
            "top_20": np.nanpercentile(values, 80),
            "top_25": np.nanpercentile(values, 75),
            "top_35": np.nanpercentile(values, 65),
            "top_50": np.nanpercentile(values, 50),
            "q25": np.nanpercentile(values, 25),
            "q75": np.nanpercentile(values, 75),
            "iqr": np.nanpercentile(values, 75) - np.nanpercentile(values, 25)
            }

        # Compute statistics on top X% of the raw data
        # values_ = np.concatenate(values)
        values_ = values
        for percent in MORE_STAT_PERCENT:
            threshold = np.percentile(values_, 100 - percent)
            top_values = values_[values_ >= threshold]

            if len(top_values) > 0:
                stats.update({f"top_{percent}_mean": np.mean(top_values)})
                stats.update({f"top_{percent}_median": np.median(top_values)})
                stats.update({f"top_{percent}_std": np.std(top_values)})
            else:
                stats[f"top_{percent}_mean"] = np.nan
                stats[f"top_{percent}_median"] = np.nan
                stats[f"top_{percent}_std"] = np.nan

        return stats
    except Exception as e:
        logging.error(f"Error computing additional statistics: {e}")
        return {stat: np.nan for stat in STAT_ORDER + percent_stats if stat not in STAT_ORDER[:10]}

import numpy as np
import pandas as pd

def process_raster_statistics(stats, gdf, raster_name, project_name, flight, more_stat_percent):
    """
    Processes raster statistics by extracting raw values, computing additional statistics,
    and formatting the final DataFrame with necessary metadata.

    Parameters:
        stats (list of dict): List of dictionaries containing raster statistics.
        gdf (GeoDataFrame): Geospatial dataframe containing feature IDs.
        raster_name (str): Name of the raster being processed.
        project_name (str): Project identifier.
        flight (str): Flight identifier.
        more_stat_percent (list): List of percent values for additional statistics.

    Returns:
        DataFrame: Processed statistics DataFrame.
    """
    # Extract raw values for additional calculations
    raw_values = [
        stat["mini_raster_array"].compressed() if "mini_raster_array" in stat else np.array([])
        for stat in stats
    ]

    additional_stats = [compute_additional_statistics(vals, more_stat_percent) for vals in raw_values]

    # Create DataFrame from stats and drop unwanted columns
    df_stats = pd.DataFrame(stats).drop(columns=["mini_raster_array", "mini_raster_affine", "mini_raster_nodata"], errors="ignore")
    df_stats = df_stats[STAT_ORDER[:10]]  # Assign expected column order

    # Convert additional statistics into DataFrame
    df_additional = pd.DataFrame(additional_stats)

    # Merge basic and additional statistics
    df = pd.concat([df_stats, df_additional], axis=1)

    # Prefix raster name to stat columns
    df.columns = [f"{raster_name}_{col}" for col in df.columns]

    # Insert metadata columns
    df.insert(0, "id", gdf.id)
    df.insert(1, "project", project_name)
    df.insert(2, "flight", flight)

    return df

def extract_raster_stats_multiple(shp, raster_paths, project_name, flight, output_root_folder_path, ndvi_geotiff= True, plot_geom="none"):
    """
    Extracts raster statistics for multiple raster files and saves results in a CSV file.

    Parameters:
        shp (str): Path to GeoJSON or shapefile containing geometries.
        raster_paths (dict): Dictionary of raster names and their file paths.
        project_name (str): Project name.
        flight (str): Flight name.
        output_root_folder_path (str)
        plot_geom (str, optional): Plot geometry option. Default is "none".

    Returns:
        pd.DataFrame: DataFrame with extracted raster statistics for each raster.
    """
    start_time = time.time()
    print(raster_paths)

    raster_example = raster_paths['nir']
    output_folder = os.path.join(output_root_folder_path, project_name)
    os.makedirs(output_folder, exist_ok=True)

    # Generating NDVI GEOTIFF and values
    output_ndvi = os.path.join(output_folder, f"{flight}ndvi.tif")
    # **Check if NDVI GEOTIFF for the flight being processed already exists**

    logging.info(f"Processing project: {project_name}, Flight: {flight}")
    logging.info(f"⚙️ Generating NDVI Orthomosaic for: {project_name}, Flight: {flight}")

    # Generating NDVI GEOTIFF
    if 'nir' and 'red' in raster_paths.keys():
        _, ndvi_values = generate_ndvi_GeoTIFF_and_values(raster_paths['nir'], raster_paths['red'], output_ndvi)
        logging.info(f"✅ Successfully generated NDVI GEOTIFF")

    else:
        logging.error(f"❌ Required rasters not found for NDVI GEOTIFF generation. NIR & Red rasters required.")

    logging.info(f"🚀 Preparing raster processing for flight: {flight}")
    logging.info(f"⚙️ Reading shapefile: {os.path.basename(shp)}")

    # Importing geojson file
    if not os.path.exists(shp):
        logging.error(f"❌ Shapefile not found: {shp}")
        raise FileNotFoundError(f"Shapefile not found: {shp}")

    try:
        gdf = gpd.read_file(shp)
        if gdf.empty:
            logging.error("❌ Shapefile is empty!")
            raise ValueError("Shapefile contains no geometries.")
        gdf.columns = gdf.columns.str.lower()  # Convert column names to lowercase

        # Ensure 'id' is numeric and sort geodataframe on 'id' column
        if pd.to_numeric(gdf["id"], errors="coerce").notna().all():
            gdf = gdf.sort_values(by="id", ascending=True).reset_index(drop=True)

        # Ensure CRS alignment between gdf and one of the rastesrs
        with rasterio.open(raster_example) as src:
            gdf = ensure_crs_alignment(gdf, src.crs)

    except Exception as e:
        logging.error(f"Error reading shapefile: {e}")
        raise

    all_results = []

    # Compute NDVI stats
    # zonal_stats requires spatial information to correctly match geometries with raster values.
    # When using a file path, it reads the affine transformation automatically.
    # When passing a NumPy array, we must manually specify the affine transformation.
    logging.info(f"⚙️ Processing NDVI Stats")

    try:
        # We can use original affine from one of the rasters
        with rasterio.open(raster_example) as src:
            original_affine = src.transform
        stats = zonal_stats(
            gdf, ndvi_values,
            affine=original_affine,
            stats=STAT_ORDER[:10],  # Extracting basic stats
            raster_out=True  # Extract raw pixel values for additional stats
        )

        processed_df = process_raster_statistics(stats, gdf, 'NDVI', project_name, flight, MORE_STAT_PERCENT)
        all_results.append(processed_df)
        logging.info(f"✅ Successfully processed: NDVI Stats")
    except:
        logging.error(f"❌ Error generating NDVI Statictics.")

    # Compute stats for rasters
    for raster_name in raster_paths.keys():
        if raster_name not in raster_paths:
            continue
        raster_path = raster_paths[raster_name]
        logging.info(f"⚙️ Processing raster: {raster_name} ({os.path.basename(raster_path)})")

        # Validate raster file existence
        if not os.path.exists(raster_path):
            logging.error(f"Raster file not found: {raster_path}")
            continue  # Skip missing rasters instead of stopping execution

        with rasterio.open(raster_path) as src:
            gdf = ensure_crs_alignment(gdf, src.crs)

            stats = zonal_stats(
                gdf, raster_path,
                stats=STAT_ORDER[:10],  # Extracting basic stats
                raster_out=True  # Extract raw pixel values for additional stats
            )
        processed_df = process_raster_statistics(stats, gdf, raster_name, project_name, flight, MORE_STAT_PERCENT)

        all_results.append(processed_df)
        logging.info(f"✅ Successfully processed: {raster_name}")

    # Merge all results
    if not all_results:
        # logging.error("❌ No raster statistics could be extracted.")
        return pd.DataFrame()

    final_df = pd.concat(all_results, axis=1).loc[:, ~pd.concat(all_results, axis=1).columns.duplicated()]

    percent_stats = list(itertools.chain.from_iterable([[f"top_{percent}_mean", f"top_{percent}_median", f"top_{percent}_std"] for percent in MORE_STAT_PERCENT]))


    output_file = os.path.join(output_folder, f"{flight}statistics.csv")

    final_df.to_csv(output_file, index=False)

    logging.info(f"✅📌 Statistics saved to: {output_file}")
    print(f"✅📌 Statistics saved to: {output_file}")
    print(f"⏳ Total execution time: {time.time() - start_time:.2f} seconds")

    return final_df




# Example use for singel project

In [42]:
# # Example Usage
# src_folder = r'D:\PhenoCrop\2_pix4d_cleaned\E166\MS\20240812 E166 M3M 30m MS 80 85\2_Orthomosaics'
# paths = find_files_in_folder(src_folder, 'tif')
# raster_files = find_raster_files(paths, RASTER_ORDER)
# # shapefile_path = r"D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 E 166_test.geojson"
# shapefile_path = r"D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 E166_sorted_ID_corrected_coordinate_system_polygons_shrinked.geojson"
# project = "E166"
# flight = "20240812 E166 M3M 30m MS 80 85"
# output_folder = r"D:\PhenoCrop\3_python\test"

# stats_df = extract_raster_stats_multiple(shapefile_path, raster_files, project, flight, output_folder)

# Processing multiple fields

In [119]:
import os
import re
import glob
import gc
import psutil  # To monitor memory usage
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from shapely.geometry import box

def extract_common_prefix(file_paths):
    """Finds the common prefix among a list of file names."""
    common_prefix = os.path.commonprefix([os.path.basename(p) for p in file_paths])
    return re.sub(r'[^a-zA-Z0-9_-]', '', common_prefix.replace(" ", "_"))  # Clean any special characters

def list_geojson_files(geojson_file_folder):
    """
    Lists all geojson files in the specified folder and extracts project names from their filenames.

    Parameters:
    - geojson_file_folder (str): Path to the folder containing geojsonfiles.

    Returns:
    - dict: A dictionary mapping project names to their corresponding geojson file paths.
    """
    geojson_file_dict = {}
    geojson_files = glob.glob(os.path.join(geojson_file_folder, "*.geojson"))

    for shp in geojson_files:
        project_name = os.path.basename(shp).split(".geojson")[0]
        geojson_file_dict[project_name] = shp  # Map project name to geojsonfile path

    return geojson_file_dict

def check_memory_usage(threshold=80):
    """
    Checks memory usage and prints a warning if it exceeds the threshold.
    
    Parameters:
    - threshold (int): Memory usage percentage above which a warning is triggered.
    
    Returns:
    - bool: True if memory usage is high, False otherwise.
    """
    mem = psutil.virtual_memory()
    if mem.percent > threshold:
        print(f"⚠️ WARNING: High memory usage detected ({mem.percent}%). Consider freeing memory.")
        return True
    return False

@precise_timing_decorator
def prepare_and_run_raster_processing(project_name, output_root_folder, ortho_dict, geojson_file_folder):
    """
    Matches orthomosaics to the correct geojson file and prepares necessary data before running process_rasters().

    Parameters:
    - project_name (str): Name of the field project
    - output_root_folder (str): Root path to store output CSV files.
    - ortho_dict (dict): Dictionary mapping flight folders to lists of raster file paths.
    - geojson_file_folder (str): Path to the folder containing geojsonfiles.

    Returns:
    - None
    """
    geojson_file_dict = list_geojson_files(geojson_file_folder)

    for flight, paths in ortho_dict.items():
        print(f"\n🚀 Preparing raster processing for flight: {flight}")

        # Find the corresponding geojsonfile
        matched_geojson_file = next(
            (shp for shp_file_name, shp in geojson_file_dict.items() if project_name in shp_file_name), None
        )

        if not matched_geojson_file:
            print(f"⚠️ No matching geojson file found for {flight}. Skipping...")
            continue

        print(f"✅ Matched geojson file: {matched_geojson_file}")

        
        # Define output file name based on the common prefix of input files
        output_csv_name = f"{extract_common_prefix(paths)}statistics.csv"
        output_csv_path = os.path.join(output_root_folder, project_name, output_csv_name)
        
        # Ensure the the output directory exists
        os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
        
        # **Check if CSV file for the flight being processed already exists**
        if os.path.exists(output_csv_path):
            print(f"📌 Output CSV already exists: {output_csv_path}. Skipping processing.")
            continue  # Skip this flight
        
        # **Monitor memory before loading large files**
        check_memory_usage()

        # Identify rgb raster path dynamically
        rgb_raster_path = next((tif for tif in paths if "_transparent_mosaic_group1.tif" in tif), None)

        # Downscale the RGB raster for preview during processing
        if rgb_raster_path:
            rgb_raster_thumb_path = add_suffix_to_file_name(rgb_raster_path, "_thumb")
            if os.path.exists(rgb_raster_thumb_path):
                print(f"📌 Resized RGB Ortho exists: {rgb_raster_thumb_path}. Skipping downscaling.")
            else:
                downscale_image(rgb_raster_path, rgb_raster_thumb_path, 0.25)

        # Generate dick of raster file paths
        raster_files = find_raster_files(paths, RASTER_ORDER)

        # **Check memory usage after loading**
        check_memory_usage()

        stats_df = extract_raster_stats_multiple(matched_geojson_file, raster_files, project_name, extract_common_prefix(paths), output_root_folder)


        # plot_geom = "ortho"
        # plot_geom = "shp"
        # plot_geom = "none"

        # **Clear raster data from memory**
        gc.collect()  # Force garbage collection
        print(f"🧹 Cleared raster data from memory.")

        gc.collect()  # Force garbage collection
        print(f"🧹 Cleared shapefile data from memory.")

        # Save results
        # stats_df.to_csv(output_csv_path, index_label="id")
        print(f"📁 Saved results to {output_csv_path}")

        # **Final memory check**
        check_memory_usage()

# if __name__ == "__main__":
#     geojson_file_folder = r'D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON'
#     output_root_folder = r'D:\PhenoCrop\3_python'
#     prepare_and_run_raster_processing(project_name, output_root_folder, ortho_dict, geojson_file_folder)


In [121]:
import os

def process_multiple_projects(project_names, src_folder, flight_type, geojson_file_folder, output_root_folder):
    """
    Processes multiple projects by fetching orthomosaic files, validating data, and running raster analysis.

    Parameters:
    - project_names (list of str): List of project names to process.
    - src_folder (str): Root folder containing project data.
    - flight_type (str): Either 'MS' or '3D' indicating the flight type.
    - geojson_file_folder (str): Path to folder containing GeoJSON shapefiles.
    - output_root_folder (str): Folder where processed CSV files will be saved.

    Returns:
    - None
    """
    for project_name in project_names:
        print(f"\n🚀 Processing Project: {project_name} | Flight Type: {flight_type}")

        # **Step 1: Generate ortho_dict**
        ortho_dict, dsm_dtm_dict = get_project_files(src_folder, project_name, flight_type)



        completeness, missing = check_data_completeness(ortho_dict, dsm_dtm_dict)

        print("\n✅ COMPLETENESS STATUS:")
        for flight, status in completeness.items():
            print(f"{flight}: {status}")

        if missing:
            print("\n❌ MISSING FILES REPORT:")
            for flight, files in missing.items():
                print(f"{flight} is missing: {', '.join(files)}")

        # Drop incomplete flights
        ortho_dict, dsm_dtm_dict = drop_incomplete_flights(ortho_dict, dsm_dtm_dict, missing, output_root_folder)

        # **Step 4: Run raster processing**
        prepare_and_run_raster_processing(project_name, output_root_folder, ortho_dict, geojson_file_folder)

if __name__ == "__main__":
    # Example usage
    field_ids = ['E166', 'PRO_BAR_VOLL', 'OAT_FRONTIERS', 'DIVERSITY_OATS', 'PRO_BAR_SØRÅS', 'PHENO_CROP', 'PILOT']
    src_folder = r"D:\PhenoCrop\2_pix4d_cleaned"
    project_names = [field for field in field_ids if field in os.listdir(src_folder)]   # List of projects
    # project_names_TEST = ['E166']   # TEMP List of projects
    geojson_file_folder = r'D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON'
    output_root_folder = r'D:\PhenoCrop\3_python\V2'
    print(project_names)
    # for flight_type in ["MS", "3D"]:  # Process both flight types
    for flight_type in ["MS"]:  # Process only MS flight types
        process_multiple_projects(project_names, src_folder, flight_type, geojson_file_folder, output_root_folder)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\PhenoCrop\\2_pix4d_cleaned'

In [195]:
import os

def process_multiple_projects(project_names, src_folder, flight_type, geojson_file_folder, output_root_folder):
    """
    Processes multiple projects by fetching orthomosaic files, validating data, and running raster analysis.

    Parameters:
    - project_names (list of str): List of project names to process.
    - src_folder (str): Root folder containing project data.
    - flight_type (str): Either 'MS' or '3D' indicating the flight type.
    - geojson_file_folder (str): Path to folder containing GeoJSON shapefiles.
    - output_root_folder (str): Folder where processed CSV files will be saved.

    Returns:
    - None
    """
    for project_name in project_names:
        print(f"\n🚀 Processing Project: {project_name} | Flight Type: {flight_type}")

        # **Step 1: Generate ortho_dict**
        ortho_dict, dsm_dtm_dict = get_project_files(src_folder, project_name, flight_type)



        completeness, missing = check_data_completeness(ortho_dict, dsm_dtm_dict)

        print("\n✅ COMPLETENESS STATUS:")
        for flight, status in completeness.items():
            print(f"{flight}: {status}")

        if missing:
            print("\n❌ MISSING FILES REPORT:")
            for flight, files in missing.items():
                print(f"{flight} is missing: {', '.join(files)}")

        # Drop incomplete flights
        ortho_dict, dsm_dtm_dict = drop_incomplete_flights(ortho_dict, dsm_dtm_dict, missing, output_root_folder)
        
        ortho_dict = {key: ortho_dict[key] for key in incomplete}

        # **Step 4: Run raster processing**
        prepare_and_run_raster_processing(project_name, output_root_folder, ortho_dict, geojson_file_folder)
        return ortho_dict

if __name__ == "__main__":# Example usage
    field_ids = ['MASBASIS_2022']
    src_folder = r"D:\V-Pheno\2_pix4d_cleaned"
    project_names = [field for field in field_ids if field in os.listdir(src_folder)]   # List of projects
    # project_names_TEST = ['E166']   # TEMP List of projects
    geojson_file_folder = r'D:\V-Pheno'
    output_root_folder = r'D:\V-Pheno\5_1_stats_vollebekk_python_2'
    print(project_names)
    # for flight_type in ["MS", "3D"]:  # Process both flight types
    for flight_type in ["MS"]:  # Process only MS flight types
        ooro = process_multiple_projects(project_names, src_folder, flight_type, geojson_file_folder, output_root_folder)

2025-03-10 11:35:15,569 - INFO - Processing project: MASBASIS_2022, Flight: 20220822_T1106_MASBASIS_P4M_20m_MS_
2025-03-10 11:35:15,571 - INFO - ⚙️ Generating NDVI Orthomosaic for: MASBASIS_2022, Flight: 20220822_T1106_MASBASIS_P4M_20m_MS_


['MASBASIS_2022']

🚀 Processing Project: MASBASIS_2022 | Flight Type: MS
Fetching file lists for project: MASBASIS_2022, flight type: MS
⏳ Execution time of check_data_completeness: 0.000404 seconds

✅ COMPLETENESS STATUS:
masbasis-P4M-20m-MS-20220527T1025: ✅ Complete
masbasis-P4M-20m-MS-20220531T1046: ✅ Complete
masbasis-P4M-20m-MS-20220609T1207: ✅ Complete
masbasis-P4M-20m-MS-20220614T1122: ✅ Complete
masbasis-P4M-20m-MS-20220620T1038: ✅ Complete
masbasis-P4M-20m-MS-20220623T1254: ✅ Complete
masbasis-P4M-20m-MS-20220623T1333: ✅ Complete
masbasis-P4M-20m-MS-20220629T1055: ✅ Complete
masbasis-P4M-20m-MS-20220630T1138: ✅ Complete
masbasis-P4M-20m-MS-20220708T1201: ✅ Complete
masbasis-P4M-20m-MS-20220711T0933: ✅ Complete
masbasis-P4M-20m-MS-20220714T1224: ✅ Complete
masbasis-P4M-20m-MS-20220719T1239: ✅ Complete
masbasis-P4M-20m-MS-20220721T1314: ✅ Complete
masbasis-P4M-20m-MS-20220726T1034: ✅ Complete
masbasis-P4M-20m-MS-20220728T1253: ✅ Complete
masbasis-P4M-20m-MS-20220801T1225: ✅ Comp

2025-03-10 11:35:55,629 - INFO - ✅ Successfully generated NDVI GEOTIFF
2025-03-10 11:35:55,639 - INFO - 🚀 Preparing raster processing for flight: 20220822_T1106_MASBASIS_P4M_20m_MS_
2025-03-10 11:35:55,640 - INFO - ⚙️ Reading shapefile: MASBASIS_2022-2022.geojson
2025-03-10 11:35:55,764 - INFO - ⚙️ Processing NDVI Stats
2025-03-10 11:37:38,479 - INFO - ✅ Successfully processed: NDVI Stats
2025-03-10 11:37:38,479 - INFO - ⚙️ Processing raster: blue (20220822 T1106 MASBASIS P4M 20m MS_transparent_reflectance_blue.tif)
2025-03-10 11:40:23,488 - INFO - ✅ Successfully processed: blue
2025-03-10 11:40:23,488 - INFO - ⚙️ Processing raster: green (20220822 T1106 MASBASIS P4M 20m MS_transparent_reflectance_green.tif)
2025-03-10 11:43:12,316 - INFO - ✅ Successfully processed: green
2025-03-10 11:43:12,316 - INFO - ⚙️ Processing raster: nir (20220822 T1106 MASBASIS P4M 20m MS_transparent_reflectance_nir.tif)
2025-03-10 11:45:57,840 - INFO - ✅ Successfully processed: nir
2025-03-10 11:45:57,842 - 

✅📌 Statistics saved to: D:\V-Pheno\5_1_stats_vollebekk_python_2\MASBASIS_2022\20220822_T1106_MASBASIS_P4M_20m_MS_statistics.csv
⏳ Total execution time: 975.31 seconds
🧹 Cleared raster data from memory.
🧹 Cleared shapefile data from memory.
📁 Saved results to D:\V-Pheno\5_1_stats_vollebekk_python_2\MASBASIS_2022\20220822_T1106_MASBASIS_P4M_20m_MS_statistics.csv
⏳ Execution time of prepare_and_run_raster_processing: 976.039425 seconds


In [177]:
import os

def process_multiple_projects(project_names, src_folder, flight_type, geojson_file_folder, output_root_folder):
    """
    Processes multiple projects by fetching orthomosaic files, validating data, and running raster analysis.

    Parameters:
    - project_names (list of str): List of project names to process.
    - src_folder (str): Root folder containing project data.
    - flight_type (str): Either 'MS' or '3D' indicating the flight type.
    - geojson_file_folder (str): Path to folder containing GeoJSON shapefiles.
    - output_root_folder (str): Folder where processed CSV files will be saved.

    Returns:
    - None
    """
    for project_name in project_names:
        print(f"\n🚀 Processing Project: {project_name} | Flight Type: {flight_type}")

        # **Step 1: Generate ortho_dict**
        ortho_dict, dsm_dtm_dict = get_project_files(src_folder, project_name, flight_type)

        print('here',ortho_dict)
        completeness, missing = check_data_completeness(ortho_dict, dsm_dtm_dict)
        
        print("\n✅ COMPLETENESS STATUS:")
        for flight, status in completeness.items():
            print(f"{flight}: {status}")

        if missing:
            print("\n❌ MISSING FILES REPORT:")
            for flight, files in missing.items():
                print(f"{flight} is missing: {', '.join(files)}")

        # Drop incomplete flights
        ortho_dict, dsm_dtm_dict = drop_incomplete_flights(ortho_dict, dsm_dtm_dict, missing, output_root_folder)

        # **Step 4: Run raster processing**
        # prepare_and_run_raster_processing(project_name, output_root_folder, ortho_dict, geojson_file_folder)

if __name__ == "__main__":# Example usage
    field_ids = ['MASBASIS_2022']
    src_folder = r"D:\V-Pheno\2_pix4d_cleaned\MASBASIS_2022"
    project_names = [field for field in field_ids if field in os.listdir(src_folder)]   # List of projects
    # project_names_TEST = ['E166']   # TEMP List of projects
    geojson_file_folder = r'D:\V-Pheno'
    output_root_folder = r'D:\V-Pheno\5_1_stats_vollebekk_python_2'
    print(project_names)
    # for flight_type in ["MS", "3D"]:  # Process both flight types
    for flight_type in ["MS"]:  # Process only MS flight types
        print(flight_type, project_names)
        process_multiple_projects(project_names, src_folder, flight_type, geojson_file_folder, output_root_folder)

[]
MS []


In [187]:
incomplete = ['masbasis-P4M-20m-MS-20220822T1106']

In [18]:
# # Example Usage
# src_folder = r'D:\PhenoCrop\2_pix4d_cleaned\E166\MS\20240812 E166 M3M 30m MS 80 85\2_Orthomosaics'
# paths = find_files_in_folder(src_folder, 'tif')
# raster_files = find_raster_files(paths, RASTER_ORDER)
# # shapefile_path = r"D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 E 166_test.geojson"
# shapefile_path = r"D:\PhenoCrop\3_qgis\3_Extraction Polygons\3. FINAL MASKS PYTHON\24 E166_sorted_ID_corrected_coordinate_system_polygons_shrinked.geojson"
# project = "E166"
# flight = "20240812 E166 M3M 30m MS 80 85"
# output_folder = r"D:\PhenoCrop\3_python\test"

# stats_df = extract_raster_stats_multiple(shapefile_path, raster_files, project, flight, output_folder)